# Importing the neccessary libraries

In [7]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Importing the dataset

In [22]:
# Download data from yfinance
ticker_symbol = 'SRF.NS'  # Example: Apple Inc.
start_date = '2010-01-01'
end_date = '2023-01-01'
data = yf.download(ticker_symbol, start=start_date, end=end_date)

# Display the first few rows of the data
print(data.head())

# Data preprocessing steps
data.index = pd.to_datetime(data.index)  # Convert index to datetime
data.dropna(inplace=True)  # Handle missing values

[*********************100%%**********************]  1 of 1 completed

                 Open       High        Low      Close  Adj Close   Volume
Date                                                                      
2010-01-04  41.700001  42.740002  40.959999  42.400002  32.080799  2526145
2010-01-05  42.779999  44.389999  42.610001  42.900002  32.459103  5181960
2010-01-06  43.000000  43.980000  42.439999  42.740002  32.338043  1977080
2010-01-07  42.959999  43.580002  42.320000  42.560001  32.201855  1925005
2010-01-08  42.759998  43.500000  42.500000  42.669998  32.285076  1626465


# Making Sequences

In [23]:
# Function to create sequences
def create_sequences(data, seq_length):
    xs, ys = [], []
    for i in range(len(data)-seq_length):
        x = data[i:i+seq_length]
        y = data[i+seq_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

# Define sequence length
sequence_length = 10

In [24]:
# Initialize dictionaries to hold data
scalers = {}
X_train_dict, X_test_dict = {}, {}
y_train_dict, y_test_dict = {}, {}
features = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']

# Scale data and create sequences for each feature
for feature in features:
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data[feature].values.reshape(-1, 1))
    scalers[feature] = scaler

    X, y = create_sequences(scaled_data, sequence_length)

    split_ratio = 0.8
    split_index = int(split_ratio * len(X))

    X_train_dict[feature], X_test_dict[feature] = X[:split_index], X[split_index:]
    y_train_dict[feature], y_test_dict[feature] = y[:split_index], y[split_index:]

# Build and Train LSTM models

In [25]:
# Function to build and train LSTM model
def build_and_train_model(X_train, y_train, sequence_length):
    model = Sequential([
        LSTM(units=100, activation='relu', input_shape=(sequence_length, 1)),
        Dense(units=1)
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    history = model.fit(X_train, y_train, epochs=200, batch_size=32, validation_split=0.2, verbose=0)
    return model, history

# Train models for each feature
models, histories = {}, {}
for feature in features:
    start_time = time.time()
    X_train, y_train = X_train_dict[feature], y_train_dict[feature]
    model, history = build_and_train_model(X_train, y_train, sequence_length)
    models[feature] = model
    histories[feature] = history
    print(f'Model trained for {feature}')
    print(f'Time taken: {time.time()-start_time:.2f} seconds\n')

/home/anupam/miniconda3/envs/myenv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Model trained for Open
Time taken: 165.94 seconds

Model trained for High
Time taken: 161.41 seconds

Model trained for Low
Time taken: 152.50 seconds

Model trained for Close
Time taken: 155.62 seconds

Model trained for Adj Close
Time taken: 156.49 seconds

Model trained for Volume
Time taken: 150.52 seconds



# Making Predictions

In [26]:
# Make predictions on the test set and inverse transform them
y_pred_dict, y_test_inverse_dict = {}, {}
for feature in features:
    model = models[feature]
    scaler = scalers[feature]
    X_test, y_test = X_test_dict[feature], y_test_dict[feature]

    y_pred = model.predict(X_test)
    y_pred_inverse = scaler.inverse_transform(y_pred)
    y_test_inverse = scaler.inverse_transform(y_test.reshape(-1, 1))

    y_pred_dict[feature] = y_pred_inverse
    y_test_inverse_dict[feature] = y_test_inverse

20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step  
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


# Visualising the results

In [27]:
import plotly.graph_objects as go

# Function to create a candlestick plot
def plot_candlestick(actual_data, predicted_data, title):
    fig = go.Figure(data=[go.Candlestick(x=actual_data.index,
                                         open=actual_data['Open'],
                                         high=actual_data['High'],
                                         low=actual_data['Low'],
                                         close=actual_data['Close'],
                                         name='Actual'),
                          go.Candlestick(x=actual_data.index,
                                         open=predicted_data['Open'],
                                         high=predicted_data['High'],
                                         low=predicted_data['Low'],
                                         close=predicted_data['Close'],
                                         name='Predicted',
                                         increasing_line_color='cyan',
                                         decreasing_line_color='blue')])

    fig.update_layout(title=title, xaxis_title='Date', yaxis_title='Price')
    fig.show()

# Prepare actual and predicted data for plotting
actual_data = data.iloc[-len(y_test_inverse_dict['Close']):]
predicted_data = pd.DataFrame(index=actual_data.index)

for feature in features:
    predicted_data[feature] = y_pred_dict[feature].flatten()

# Plot the candlestick chart
plot_candlestick(actual_data, predicted_data, 'Actual vs Predicted Stock Prices')